In [7]:
import pandas as pd
import re
import numpy as np

from anndata import AnnData



In [8]:
from mapping import generate_gene_id_name_map 

gene_id_name_map, gene_name_id_map = generate_gene_id_name_map()

Extracted 43945 mappings from 73467 ensembl lines
Extracted 38606 mappings from gene id gtf file
Total Mappings Extracted 59979


In [ ]:

gtf_file = '/nfs/turbo/umms-indikar/shared/projects/reference_genome/prebuilt/refdata-gex-GRCh38-2024-A/genes/genes.gtf'

df = pd.read_csv(gtf_file, sep='\t', header=None, comment='#')

df_back = df.copy()

df = df[df[2] == 'gene']


In [ ]:
def format_gtf(dfc):
    df = dfc.copy()
    df.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
    
    fields = ['gene_id', 'transcript_id', 'exon_number', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_name', 'transcript_source', 'transcript_biotype', 'protein_id', 'exon_id', 'tag']
    for field in fields:
        df[field] = df['attribute'].apply(lambda x: re.findall(rf'{field} "([^"]*)"', x)[0] if rf'{field} "' in x else '')
    
    df.replace('', np.nan, inplace=True)
    
    df.drop('attribute', axis=1, inplace=True)
    return df

In [ ]:
formatted_df = format_gtf(df)
mapping_dict = dict(zip(formatted_df['gene_id'], formatted_df['gene_name']))

formatted_df[['gene_id', 'gene_name']].to_csv('./data/gene_id_name_map_gtf.csv')

In [5]:

gene_metadata = pd.read_csv('./data/DSET038_37_Combined_GRCh38p14_GRCh37p13.txt')
gene_metadata

### B Matrix Operations

In [4]:
B_MATRIX_PATH = '/scratch/indikar_root/indikar1/shared_data/HWG/operations'

X = pd.read_csv(f'{B_MATRIX_PATH}/DSET051_B_Matrix_TFsONLY_TFTGDb_TFLink_ChEA3_X.csv', header=None, index_col=None)
X_arr = X.to_numpy()
obs = pd.read_csv(f'{B_MATRIX_PATH}/DSET051_B_Matrix_TFsONLY_TFTGDb_TFLink_ChEA3_obs.csv')
var = pd.read_csv(f'{B_MATRIX_PATH}/DSET051_B_Matrix_TFsONLY_TFTGDb_TFLink_ChEA3_var.csv')

In [5]:
adata = AnnData(X=X_arr, obs=obs, var=var)

/home/nuvi/.local/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/nuvi/.local/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
adata.obs

,TFStableIDVersion,TFStableID,TFName,TFClass,IsTF,GeneVersion,GeneDescription,Entrez_Description,GeneType,ChromosomescaffoldName,...,Motif_Status,ORFeome_Activator,DelRosso_Activators,DelRosso_Repressors,DelRosso_Bifunctional,TRRUST_Activators,TRRUST_Repressors,TRRUST_Conflicting,Sums,Target_Gene_Count
0,ENSG00000001167.15,ENSG00000001167,NFYA,Activator,Yes,15,DNA binding; regulation of DNA-templated trans...,nuclear transcription factor Y subunit alpha,protein_coding,6,...,In vivo/Misc source,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1,15524
1,ENSG00000004848.8,ENSG00000004848,ARX,Repressor,Yes,8,DNA binding; regulation of DNA-templated trans...,aristaless related homeobox,protein_coding,X,...,High-throughput in vitro,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1,402
2,ENSG00000005073.6,ENSG00000005073,HOXA11,Repressor,Yes,6,DNA binding; regulation of DNA-templated trans...,homeobox A11,protein_coding,7,...,High-throughput in vitro,NaN,NaN,-1.0,NaN,NaN,-1.0,NaN,-2,381
3,ENSG00000005102.14,ENSG00000005102,MEOX1,Conflicted,Yes,14,DNA binding; regulation of DNA-templated trans...,mesenchyme homeobox 1,protein_coding,17,...,High-throughput in vitro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,977
4,ENSG00000005513.11,ENSG00000005513,SOX8,Conflicted,Yes,11,organelle; nucleus; DNA binding; reproductive ...,SRY-box transcription factor 8,protein_coding,16,...,High-throughput in vitro,NaN,1.0,-1.0,NaN,NaN,NaN,NaN,0,5308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,ENSG00000277258.5,ENSG00000277258,PCGF2,Repressor,Yes,5,organelle; nucleus; nucleoplasm; DNA binding; ...,polycomb group ring finger 2,protein_coding,17,...,No motif,NaN,NaN,-1.0,NaN,NaN,-1.0,NaN,-2,3022
1604,ENSG00000277462.2,ENSG00000277462,ZNF670,Repressor,Yes,2,regulation of DNA-templated transcription; org...,zinc finger protein 670,protein_coding,1,...,No motif,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1,458
1605,ENSG00000278129.2,ENSG00000278129,ZNF8,Conflicted,Yes,2,regulation of DNA-templated transcription; org...,zinc finger protein 8,protein_coding,19,...,High-throughput in vitro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1171
1606,ENSG00000278318.5,ENSG00000278318,ZNF229,Conflicted,Yes,5,regulation of DNA-templated transcription; org...,zinc finger protein 229,protein_coding,19,...,No motif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,456


In [54]:
adata.write(f'{B_MATRIX_PATH}/DSET051_B_Matrix_TFsONLY_TFTGDb_TFLink_ChEA3.h5ad')

In [16]:
tenk_b_matrix = pd.read_csv('/nfs/turbo/umms-indikar/shared/projects/DGC/data/processed_data/10kbp_up_10kbp_down_B.csv')

five_h_b_matrix = pd.read_csv('/nfs/turbo/umms-indikar/shared/projects/DGC/data/processed_data/500bp_up_100bp_down_B.csv')

tenk_b_matrix['GeneName'] = tenk_b_matrix['Unnamed: 0']
tenk_b_matrix.drop('Unnamed: 0', axis=1, inplace=True)

five_h_b_matrix['GeneName'] = five_h_b_matrix['Unnamed: 0']
five_h_b_matrix.drop('Unnamed: 0', axis=1, inplace=True)


In [17]:
from mapping import get_sorted_gene_order

flat_col_order = get_sorted_gene_order()

nodes = flat_col_order
nodenames = [gene_id_name_map.get(node, 'NA') for node in nodes]
len(nodes)
len(nodenames)

ordering 18581 genes


18581

In [18]:
len(set(adata.var['GeneStableID']).intersection(flat_col_order))

18251

In [21]:
# b_gene_names = set(tenk_b_matrix['GeneName'])


b_gene_names = set(flat_col_order)

b_gene_names = set(nodenames)

tf_names = set(tenk_b_matrix.keys())

tfnames = set(adata.obs['TFName'])
genes = set(adata.var['GeneName'])


print(len(tf_names.intersection(tfnames)))
print(len(b_gene_names.intersection(genes)))

print(len(tf_names.union(tfnames)))
print(len(b_gene_names.union(genes)))

print(len(b_gene_names))
print(len(tf_names))


1418
18227
1796
19876
18358
1606


In [111]:
tfnames = adata.obs['TFName']
genenames = adata.var['GeneName']

tenk_aligned = tenk_b_matrix.reindex(index=tfnames, columns=genenames, fill_value=0)
five_b_aligned = five_h_b_matrix.reindex(index=tfnames, columns=genenames, fill_value=0)

adata.layers['10kp_up_10kb_down'] = tenk_aligned.to_numpy()
adata.layers['500bp_up_100bp_down'] = five_b_aligned.to_numpy()

tenk_aligned


GeneName,ACADM,CNST,DYRK3,TTF2,GREM2,ARNT,PHACTR4,ATF3,ATP1A4,ATP1B1,...,OR51A4,NPIPB12,IGKV1D-16,UPK3BL2,IGKV1-9,IGHG4,CXorf51A,FAM236C,DEFB130B,TAF11L13
TFName,,,,,,,,,,,,,,,,,,,,,
NFYA,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
ARX,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
HOXA11,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
MEOX1,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
SOX8,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCGF2,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZNF670,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZNF8,0,0,0,0,0,0.0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
print_stats(tenk_aligned, 'tenk_aligned')

number of non zero interactions for threshold 1.5 and tenk_aligned region: 0.00%
number of zero interactions for threshold 1.5 and tenk_aligned region: 1.00%


In [103]:
threshold = 1.5

tenk_pure_mat = tenk_b_matrix.select_dtypes(include='number')

five_h_pure_mat = five_h_b_matrix.select_dtypes(include='number')

thresh_tenk_b_matrix = tenk_pure_mat[tenk_pure_mat > threshold]

thresh_five_h_b_matrix = five_h_pure_mat[five_h_pure_mat > threshold]

binary_tenk_b_matrix = (thresh_tenk_b_matrix > threshold).astype(int)
binary_five_h_b_matrix = (thresh_five_h_b_matrix > threshold).astype(int)

In [105]:

def print_stats(df, label):
    
    non_zero_count = (df != 0).sum().sum()
    zero_count = (df == 0).sum().sum()
    total = df.size
    
    non_zero_percent = non_zero_count / total
    zero_percent = zero_count / total


    print(f"number of non zero interactions for threshold {threshold} and {label} region: {non_zero_percent:.2f}%")
    print(f"number of zero interactions for threshold {threshold} and {label} region: {zero_percent:.2f}%")

df = binary_tenk_b_matrix
print_stats(df, 'larger')

df = binary_five_h_b_matrix
print_stats(df, 'smaller')


number of non zero interactions for threshold 1.5 and larger region: 0.75%
number of zero interactions for threshold 1.5 and larger region: 0.25%
number of non zero interactions for threshold 1.5 and smaller region: 0.21%
number of zero interactions for threshold 1.5 and smaller region: 0.79%


In [84]:
binary_tenk_b_matrix['GeneName'] = tenk_b_matrix['Unnamed: 0']
binary_five_h_b_matrix['GeneName'] = five_h_b_matrix['Unnamed: 0']

In [92]:
adata.write('/scratch/indikar_root/indikar1/shared_data/HWG/operations/B_matrices.h5ad')

(27090, 1606)

In [106]:
b_prime_2_0 = pd.DataFrame(adata.X, index=adata.obs[''], columns=adata.var_names)

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1608, 19782))

In [5]:
from mapping import get_sorted_gene_order

flat_col_order = get_sorted_gene_order()

nodes = flat_col_order
len(nodes)

ordering 18582 genes


18582

In [ ]:


import networkx as nx


G = nx.from_pandas_edgelist(A_matrix_edgelist, source='source', target='target', edge_attr='weight', create_using=nx.DiGraph())

adj_matrix = nx.to_scipy_sparse_array(G, nodelist=sorted(G.nodes()), weight='weight', format='csr')

adata = AnnData(X=adj_matrix)
adata.obs_names = nodes
adata.var_names = nodes

adata.write('/scratch/indikar_root/indikar1/shared_data/HWG/data/HWG/A_matrix.h5ad')

In [3]:
from constants import HWG_BASE_PATH
import os

from mapping import get_string_protein_gene_map, get_TF_set
from constants import HARD_WIRED_GENOME_A_EDGELIST, STRING_EXTRACTED_URL_PATH, HI_UNION_PATH, LIT_BM_PATH


In [10]:

threshold = 0

if not os.path.exists(HWG_BASE_PATH):
    os.makedirs(HWG_BASE_PATH)
print("Constructing Hard Wired Genome A Matrix")

if nodes is None:
    nodes = get_sorted_gene_order()

# with open(ENSEMBL_PROTEIN_GENE_PATH, "r") as protein_gene_file:
#     protein_gene_map = {}
#     for line in protein_gene_file:
#         gene, protein = line.split()
#         protein_gene_map[protein] = gene

string_protein_gene_map = get_string_protein_gene_map()

n = len(nodes)

node_idx = {g: i for i, g in enumerate(nodes)}
adj = np.zeros((len(nodes), len(nodes)), dtype=int)

# adj_df = pd.DataFrame(0, index=nodes, columns=nodes)

# initialize edge set
str_edges = set()
count = 0

with open(STRING_EXTRACTED_URL_PATH, "r") as f: 
    string_links = f.readlines()

    # skipping header line
    for link in string_links[1:]:
        count += 1
        p1, p2, score = link.strip().split(' ')
        p1 = p1.split('.')[1]
        p2 = p2.split('.')[1]
        score = int(score)
        g1 = string_protein_gene_map.get(p1, None)
        g2 = string_protein_gene_map.get(p2, None)
        # if score >= threshold:
        
        # if g1 in nodes and g2 in nodes:
        if g1 is not None and g2 is not None and score >= threshold:
            str_edges.add((g1, g2, score))
print("STRING interactions processed: ", len(str_edges))




Constructing Hard Wired Genome A Matrix
STRING interactions processed:  13715404


In [31]:

huri_edges = set()
with open(HI_UNION_PATH, "r") as huri_links:
    for link in huri_links:
        g1, g2 = link.strip().split()
        huri_edges.add((g1, g2, 1))
print("HI-UNION interactions processed: ", len(huri_edges))
huri_count = len(huri_edges)

with open(LIT_BM_PATH, "r") as lit_links:
    for link in lit_links:
        g1, g2 = link.strip().split()
        huri_edges.add((g1, g2, 1))
print("LIT-BM interactions processed: ", len(huri_edges) - huri_count)

tflist = get_TF_set()
print(tflist[:10])

# for g1, g2, inter in edges:
#     if g1 in node_idx and g2 in node_idx:
#         i, j = node_idx[g1], node_idx[g2]
#         adj[i, j] = 1


HI-UNION interactions processed:  64006
LIT-BM interactions processed:  12578
processed 1639 Transcription Factors
['DUX1_HUMAN', 'DUX3_HUMAN', 'ENSG00000001167', 'ENSG00000004848', 'ENSG00000005073', 'ENSG00000005102', 'ENSG00000005513', 'ENSG00000005801', 'ENSG00000005889', 'ENSG00000006047']


TypeError: unsupported operand type(s) for +: 'set' and 'set'

In [33]:
import pandas as pd
import networkx as nx
edges = huri_edges | str_edges
edgelist = list(edges)
edgelist_df = pd.DataFrame(edgelist, columns=['source', 'target', 'weight'])


# adj_df = pd.DataFrame(adj, index=nodes, columns=nodes)

save_path = HARD_WIRED_GENOME_A_EDGELIST.strip('.csv') + f"_{threshold}.csv"
# edgelist_df.to_csv(save_path)
A_matrix_edgelist = edgelist_df



G = nx.from_pandas_edgelist(A_matrix_edgelist, source='source', target='target', edge_attr='weight', create_using=nx.DiGraph())
G.add_nodes_from(nodes)
nodes = sorted(G.nodes())
adj_matrix = nx.to_scipy_sparse_array(G, nodelist=nodes, weight='weight', format='csr')


adata = AnnData(X=adj_matrix)
adata.obs_names = nodes
adata.var_names = nodes


NameError: name 'str_edgelest_df' is not defined

In [16]:
import anndata
import networkx as nx
adata = anndata.read_h5ad('/scratch/indikar_root/indikar1/shared_data/HWG/data/HWG/A_matrix.h5ad')

In [26]:

# STRING edges
# str_edgelist_df = pd.DataFrame(str_edges, columns=['source', 'target', 'weight'])
# G2 = nx.from_pandas_edgelist(str_edgelist_df, source='source', target='target', edge_attr='weight', create_using=nx.DiGraph())
G2.add_nodes_from(nodes)

nodes = sorted(G2.nodes())
adj_matrix = nx.to_scipy_sparse_array(G2, nodelist=nodes, weight='weight', format='csr')

adata.layers['STRING'] = adj_matrix


# HURI edges
# huri_edgelist_df = pd.DataFrame(huri_edges, columns=['source', 'target', 'weight'])
# G3 = nx.from_pandas_edgelist(huri_edgelist_df, source='source', target='target', edge_attr='weight', create_using=nx.DiGraph())
# G3.add_nodes_from(nodes)
# adj_matrix = nx.to_scipy_sparse_array(G3, nodelist=nodes, weight='weight', format='csr')

# adata.layers['HURI'] = adj_matrix

# print("Constructing Hard Wired Genome B Matrix")
# B_matrix_edgelist = A_matrix_edgelist[
# A_matrix_edgelist['source'].isin(flat_col_order) & A_matrix_edgelist['target'].isin(tflist)
# ]
# print(f'processed edges {len(B_matrix_edgelist)}')

# print("Constructing Hard Wired Genome C Matrix")
# C_matrix_edgelist = A_matrix_edgelist[
#     A_matrix_edgelist['source'].isin(tflist) & A_matrix_edgelist['target'].isin(tflist)
# ]
# print(f'processed edges {len(C_matrix_edgelist)}')


# save_path = HARD_WIRED_GENOME_B_EDGELIST.strip('.csv') + f"_{threshold}.csv"
# B_matrix_edgelist.to_csv(save_path, index=False)

# save_path = HARD_WIRED_GENOME_C_EDGELIST.strip('.csv') + f"_{threshold}.csv"
# C_matrix_edgelist.to_csv(save_path, index=False)

# print(f"Saved Hard Wired Genome to {save_path}")

In [29]:
adata.write('/scratch/indikar_root/indikar1/shared_data/HWG/data/HWG/A_matrix.h5ad')

# adata = anndata.read_h5ad('/scratch/indikar_root/indikar1/shared_data/HWG/data/HWG/A_matrix.h5ad')